In [ ]:
#예금보험기금 매매내역

#rawdata 폴더에 FOS82105 파일 필수
import os
import win32com.client
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side
import re

# 조회할 날짜 설정
specified_date = datetime.date.today()

# Outlook MAPI에 연결
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item('localoutlook')
trading_folder = root_folder.Folders['트레이딩']

# 결과를 저장할 리스트
all_results = []

# 아웃룩 메일에서 데이터 추출
for message in trading_folder.Items:
    if (message.ReceivedTime.date() == specified_date and
        ('4JM90' in message.Subject or '4JM91' in message.Subject)):
        
        body = message.HTMLBody
        soup = BeautifulSoup(body, 'html.parser')
        tables = soup.find_all('table')
        
        for table in tables:
            try:
                df = pd.read_html(StringIO(str(table)))[0]
                
                if df.shape[1] == 14:
                    df.columns = df.iloc[0]
                    df = df[1:]
                    
                    required_columns = ['운용펀드명', '종목명', '매매구분', '주문금액']
                    if all(col in df.columns for col in required_columns):
                        result_df = df[required_columns].copy()
                        result_df = result_df[result_df['운용펀드명'].notna()]
                        all_results.append(result_df)
            except:
                continue

# 펀드구분 추출 함수
def extract_fund_type(fund_name):
    if pd.isna(fund_name):
        return ""
    fund_name_str = str(fund_name)
    if '유니버스 11호' in fund_name_str:
        return '유니버스11호'
    elif '유니버스 12호' in fund_name_str:
        return '유니버스12호'
    else:
        pattern = r'유니버스\s*\d+호'
        match = re.search(pattern, fund_name_str)
        if match:
            return match.group().replace(' ', '')
        return ""

# 주문금액에서 숫자만 추출
def extract_amount(amount):
    if pd.isna(amount):
        return ""
    amount_str = str(amount).replace(',', '').replace('KRW', '').strip()
    try:
        amount_num = float(amount_str)
        if amount_num == int(amount_num):
            return int(amount_num)
        return amount_num
    except:
        return ""

# 아웃룩 메일 데이터 처리
outlook_data = []
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    
    for _, row in final_df.iterrows():
        outlook_data.append({
            '펀드구분': extract_fund_type(row['운용펀드명']),
            '거래구분': row['매매구분'] if pd.notna(row['매매구분']) else "",
            '종목명': row['종목명'] if pd.notna(row['종목명']) else "",
            '거래금리': "",
            '거래금액(액면기준)': extract_amount(row['주문금액'])
        })

# FOS82105 파일 읽기 (win32com 사용 - DRM 암호화 파일)
fos_path = r"K:\부서 공유\FI운용1부\10. 개인별폴더\김현수\rawdata"
fos_filename = f"FOS82105_{specified_date.strftime('%Y%m%d')}.xlsx"
fos_full_path = os.path.join(fos_path, fos_filename)

fos_data = []
if os.path.exists(fos_full_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False
    
    try:
        workbook = excel.Workbooks.Open(fos_full_path)
        worksheet = workbook.Worksheets(1)
        
        # 데이터가 있는 마지막 행 찾기
        last_row = worksheet.Cells(worksheet.Rows.Count, 5).End(-4162).Row  # E열 기준
        
        # 2행부터 마지막 행까지 읽기
        for row in range(2, last_row + 1):
            e_value = worksheet.Cells(row, 5).Value  # E열
            
            if e_value in ['4JM90', '4JM91']:
                fund_type = '유니버스11호' if e_value == '4JM90' else '유니버스12호'
                
                h_value = worksheet.Cells(row, 8).Value  # H열
                l_value = worksheet.Cells(row, 12).Value  # L열
                r_value = worksheet.Cells(row, 18).Value  # R열
                p_value = worksheet.Cells(row, 16).Value  # P열
                
                fos_data.append({
                    '펀드구분': fund_type,
                    '거래구분': h_value if h_value else "",
                    '종목명': l_value if l_value else "",
                    '거래금리': r_value if r_value else "",
                    '거래금액(액면기준)': p_value if p_value else ""
                })
        
        workbook.Close(False)
    finally:
        excel.Quit()

# 두 데이터 소스 병합
combined_data = outlook_data + fos_data

# 엑셀 파일 생성
wb = Workbook()
ws = wb.active

# 테두리 스타일 정의
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# 헤더 설정
headers = ['펀드구분', '거래구분', '종목명', '거래금리', '거래금액(액면기준)']
for col_num, header in enumerate(headers, 1):
    cell = ws.cell(row=1, column=col_num)
    cell.value = header
    cell.font = Font(name='맑은 고딕', size=11, bold=True)
    cell.alignment = Alignment(horizontal='center', vertical='center')
    cell.border = thin_border

# 데이터 입력
for row_num, data_row in enumerate(combined_data, 2):
    for col_num, (key, value) in enumerate(data_row.items(), 1):
        cell = ws.cell(row=row_num, column=col_num)
        cell.value = value
        cell.border = thin_border

# 열 너비 자동 조정
for column in ws.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = min(max_length + 2, 50)
    ws.column_dimensions[column_letter].width = adjusted_width

# 파일 저장
save_path = r"K:\부서 공유\FI운용1부\일일매매내역"
if not os.path.exists(save_path):
    os.makedirs(save_path)

filename = f"예금보험기금_매매내역_{specified_date.strftime('%Y%m%d')}.xlsx"
full_path = os.path.join(save_path, filename)

wb.save(full_path)
wb.close()